## This notebook providtes an exploration of using Geweke or Rubin's tests for MCMC software

This notebook uses Random Walk Metropolis targeting a the posterior of a logistic model with normal noise

In [5]:
import os
os.chdir("..")
from __future__ import print_function
import pints
import pints.toy as toy
import pints.plot
import numpy as np
import matplotlib.pyplot as plt

# Load a forward model
model = toy.LogisticModel()

# Create some toy data
real_parameters = np.array([0.015, 500])
times = np.linspace(0, 1000, 1000)
org_values = model.simulate(real_parameters, times)

# Add noise
noise = 10
values = org_values + np.random.normal(0, noise, org_values.shape)

# Create an object with links to the model and time series
problem = pints.SingleOutputProblem(model, times, values)

# Create a log-likelihood function
log_likelihood = pints.KnownNoiseLogLikelihood(problem, noise)

# Create a uniform prior over the parameters
log_prior = pints.UniformLogPrior(
    [0.01, 400],
    [0.02, 600]
)

# Create a posterior log-likelihood (log(likelihood * prior))
log_posterior = pints.LogPosterior(log_likelihood, log_prior)

# Choose starting points for 3 mcmc chains
xs = [
    real_parameters * 1.1,
    real_parameters * 0.9,
    real_parameters * 1.15,
]

# Choose a covariance matrix for the proposal step
sigma0 = np.abs(real_parameters) * 5e-4

### Geweke method

In [4]:
log_likelihood([0.015, 500])

-3666.5125679815405

In [6]:
def step(current, sigma):
    return np.random.multivariate_normal(current, sigma)
def step_accept(current, sigma)

In [40]:
log_prior(step([0.015, 500], [[0.00001, 0], [0, 10]]))

-0.69314718055994529